<a href="https://colab.research.google.com/github/AUT-Student/NLP-HW1/blob/main/NLP_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!gdown --id 16C0_9i0io43VfABV3-uukUjJYlM6k-2U
!unzip /content/HW1-datasets.zip

Downloading...
From: https://drive.google.com/uc?id=16C0_9i0io43VfABV3-uukUjJYlM6k-2U
To: /content/HW1-datasets.zip
100% 3.14M/3.14M [00:00<00:00, 16.2MB/s]
Archive:  /content/HW1-datasets.zip
replace content/HW1-datasets/train.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: content/HW1-datasets/train.txt  
replace content/HW1-datasets/valid.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: content/HW1-datasets/valid.txt  
  inflating: content/HW1-datasets/test_incomplete.txt  
  inflating: content/HW1-datasets/test.txt  
  inflating: content/HW1-datasets/test_incomplete_gold.txt  


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
file = open("/content/content/HW1-datasets/train.txt")

vocabulary = set()

for i, line in enumerate(file.readlines()):
  if i < 30*1000:
    words = line.split()

    for word in words:
      vocabulary.add(word)

In [3]:
len(vocabulary)

21188

In [4]:
sorted_vocabulary = sorted(vocabulary)

In [5]:
from bisect import bisect_left

class BigramNLM(keras.Model):
  def __init__(self, vocabulary):
    super().__init__()
    self.vocabulary = vocabulary
    self.model = keras.models.Sequential([
                                          keras.layers.Input((2,), name="Input"),
                                          keras.layers.Embedding(len(self.vocabulary), 128, name="Embedding"),
                                          keras.layers.Flatten(name="Flatten"),
                                          keras.layers.Dense(units=256, name="Hidden"),
                                          keras.layers.Dense(units=len(self.vocabulary), activation="softmax", name="Output"),
    ])
    
  def convert_word_to_index(self, word):
    pos = bisect_left(self.vocabulary, word, 0, len(self.vocabulary))
    return pos if pos != len(self.vocabulary) and self.vocabulary[pos] == word else -1
    # return self.vocabulary.index(word)

  def convert_index_to_word(self, index):
    return self.vocabulary[index]

  def call(self, inputs):
    return self.model(inputs)

  def predict(self, inputs):
    model_output =  self.model(inputs)
    predicted_word_indexes = keras.backend.argmax(model_outputs, axis=1)
    return predicted_word_indexes

In [6]:
bnlm = BigramNLM(vocabulary=sorted_vocabulary)

In [7]:
bnlm.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
file = open("/content/content/HW1-datasets/train.txt")

X_train = []
Y_train = []

for i, line in enumerate(file.readlines()):
  if i >= 30*1000:
    break

  words = line.split()

  for j in range(2, len(words)):
    word1 = words[j-2]
    word2 = words[j-1]
    word_target = words[j]

    index1 = bnlm.convert_word_to_index(word1)
    index2 = bnlm.convert_word_to_index(word2)
    index_target = bnlm.convert_word_to_index(word_target)

    X_train.append([index1, index2])
    Y_train.append(index_target)

print(len(Y_train))
X_train = np.array(X_train[:100*1000])
Y_train = tf.one_hot(indices=Y_train[:100*1000], depth=len(vocabulary)).numpy()

151278


In [10]:
len(Y_train)

50000

In [ ]:
bnlm.fit(x=X_train, y=Y_train, epochs=20)

In [ ]:
X_train = np.array([[4, 5],
           [6, 8],
           [2, 9],
           [4, 5],
           [6, 8],
           [2, 9],
           [4, 5],
           [6, 8],
           [2, 9],
           [4, 5],
           [6, 8],
           [2, 9],
           [4, 5],
           [6, 8],
           [2, 9],
           [4, 5],
           [6, 8],
           [2, 9],
           [4, 5],
           [6, 8],
           [2, 9],
           [4, 5],
           [6, 8],
           [2, 9],
           [4, 5],
           [6, 8],
           [2, 9],
           [4, 5],
           [6, 8],
           [2, 9],
           [4, 5],
           [6, 8],
           [2, 9],
           [4, 5],
           [6, 8],
           [2, 9],
           ])

# Y_train = ["سلام", "بر", "ایران","او","از","آن","که","آمد","بر"]
# Y_train = [3,2,3,4,7,5,11,2,3,3,2,3,4,7,5,11,2,3,3,2,3,4,7,5,11,2,3,3,2,3,4,7,5,11,2,3]

Y_train = tf.one_hot(indices=[3,2,3,4,7,5,11,2,3,3,2,3,4,7,5,11,2,3,3,2,3,4,7,5,11,2,3,3,2,3,4,7,5,11,2,3], depth=len(vocabulary)).numpy()

bnlm.fit(x=X_train, y=Y_train, epochs=20)

Epoch 1/20
inputs = Tensor("IteratorGetNext:0", shape=(None, 2), dtype=int64)
model_outputs = Tensor("bigram_nlm_3/sequential_3/Output/Softmax:0", shape=(None, 21188), dtype=float32)
inputs = Tensor("IteratorGetNext:0", shape=(None, 2), dtype=int64)
model_outputs = Tensor("bigram_nlm_3/sequential_3/Output/Softmax:0", shape=(None, 21188), dtype=float32)
2/2 [==============================] - 1s 154ms/step - loss: 9.9557 - accuracy: 0.0556
Epoch 2/20
2/2 [==============================] - 0s 116ms/step - loss: 9.9250 - accuracy: 0.5556
Epoch 3/20
2/2 [==============================] - 0s 111ms/step - loss: 9.8909 - accuracy: 0.5556
Epoch 4/20
2/2 [==============================] - 0s 111ms/step - loss: 9.8511 - accuracy: 0.5556
Epoch 5/20
2/2 [==============================] - 0s 117ms/step - loss: 9.7994 - accuracy: 0.5556
Epoch 6/20
2/2 [==============================] - 0s 114ms/step - loss: 9.7332 - accuracy: 0.5556
Epoch 7/20
2/2 [==============================] - 0s 110ms/step - lo

In [ ]:
keras.backend.argmax(tf.constant([
                        [4, 5],
                        [4, 7],
                        [6, 2]
                        ]), axis=1)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 1, 0])>